### Rondon's head_motion_prediction ingest and exploration

In [ ]:
import os
import sys
import plotly.express as px
import plotly.graph_objs as go
import numpy as np
from predict360user.head_motion_prediction import utils

project_path = f"predict360user/head_motion_prediction/"
sys.path.append(project_path)
from predict360user.head_motion_prediction.David_MMSys_18.Read_Dataset import (load_sampled_dataset, plot_3d_trace)

# show one traject
ONE_USER = '0'
ONE_VIDEO = '10_Cows'
if not 'SAMPLE_DATASET' in locals():
  print(f"get SAMPLE_DATASET from {project_path}")
  cwd = os.getcwd()
  os.chdir(project_path) # load_sampled_datasetuses relative path
  SAMPLE_DATASET = load_sampled_dataset()
  os.chdir(cwd)

traces = SAMPLE_DATASET[ONE_USER][ONE_VIDEO][:, 1:]

# plot 3d
plot_3d_trace(traces, ONE_USER, ONE_VIDEO)

# plot heatmap
heatmaps = []
for i in traces:
    heatmaps.append(
        utils.from_position_to_tile(
            utils.eulerian_in_range(*utils.cartesian_to_eulerian(i[0], i[1], i[2])),
            6,
            4,
        )
    )
fig = px.imshow(
    np.sum(heatmaps, axis=0),
    labels=dict(x="longitude", y="latitude", color="requests"),
    title=f"{len(traces)}_traces (reqs={np.sum(heatmaps)})",
)
fig.update_layout(go.Layout(width=600))
fig.show()


### predict360user: Ingest and exploration

In [ ]:
from predict360user import *

In [ ]:
ds = Dataset(dataset_name="all")
print(f"number of trajecs in total {ds.df.size}")
print("\nnumber of trajecs per ds:") 
print(ds.df.reset_index()["ds"].value_counts())
print("\nnumber of trajecs per entropy class:")
print(ds.df["actS_c"].value_counts())
print("\ndf head")
display(ds.df.head())

In [ ]:
# show one traject
show_traject(ds.df.loc["david","0", "10_Cows"])

In [ ]:
# entropy distribution
show_entropy_histogram(ds.df)

In [ ]:
# show representive trajectory for each entropy class
show_trajects_representative(ds.df)

In [ ]:
ds.partition()
# entropy distribution per partition
show_entropy_histogram_per_partition(ds.df)

### predict360user.Train

In [ ]:
from predict360user import *

In [ ]:
# train on david dataset
trn = Trainer(TrainerCfg(dataset_name="all", wandb_mode="disabled"))
# trn = Trainer(TrainerCfg(dataset_name="david", wandb_mode="disabled"))
trn.run()

In [ ]:
# show results
compare_train_results()
compare_eval_results()

### Visualize predictions

In [ ]:
# visualize some predictions
# # Case "most low with low"
# one_traject_low = ds.df[ds.df['actS_c'] == 'low'].nsmallest(1,'actS')
# plot = Plot360()
# plot.add_traces(one_traject_low['traces'].iloc[0])
# plot.add_predictions(one_traject_low['pos_only_low_entropy'].iloc[0])
# plot.show()

# # Case "most low with low"
# one_traject_low = ds.df[ds.df['actS_c'] == 'low'].nsmallest(1,'actS')
# plot = Plot360()
# plot.add_traces(one_traject_low['traces'].iloc[0])
# plot.add_predictions(one_traject_low['pos_only'].iloc[0])
# plot.show()

# # Case "most medium with medium"
# one_traject_medium = ds.df[ds.df['actS_c'] == 'medium'].nsmallest(1,'actS')
# plot = Plot360()
# plot.add_traces(one_traject_medium['traces'].iloc[0])
# plot.add_predictions(one_traject_medium['pos_only_medium_entropy'].iloc[0])
# plot.show()

# # Case "most medium with all"
# one_traject_medium = ds.df[ds.df['actS_c'] == 'medium'].nsmallest(1,'actS')
# plot = Plot360()
# plot.add_traces(one_traject_medium['traces'].iloc[0])
# plot.add_predictions(one_traject_medium['pos_only'].iloc[0])
# plot.show()

# # Case "most high with high"
# one_traject_high = ds.df[ds.df['actS_c'] == 'high'].nlargest(1,'actS')
# plot = Plot360()
# plot.add_traces(one_traject_high['traces'].iloc[0])
# plot.add_predictions(one_traject_high['pos_only_high_entropy'].iloc[0])
# plot.show()

# # Case "most high with all
# one_traject_high = ds.df[ds.df['actS_c'] == 'high'].nlargest(1,'actS')
# plot = Plot360()
# plot.add_traces(one_traject_medium['traces'].iloc[0])
# plot.add_predictions(one_traject_medium['pos_only'].iloc[0])
# plot.show()